In [1]:
from keras.datasets import imdb

Using TensorFlow backend.


In [2]:
voc_size = 5000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words =voc_size)

In [3]:
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Loaded dataset with 25000 training samples, 25000 test samples


In [4]:
print('-----reviews-----')
print(X_train[2])
print('----label----')
print(y_train[2])

-----reviews-----
[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 2, 54, 61, 369, 13, 71, 149, 14, 22, 112, 4, 2401, 311, 12, 16, 3711, 33, 75, 43, 1829, 296, 4, 86, 320, 35, 534, 19, 263, 4821, 1301, 4, 1873, 33, 89, 78, 12, 66, 16, 4, 360, 7, 4, 58, 316, 334, 11, 4, 1716, 43, 645, 662, 8, 257, 85, 1200, 42, 1228, 2578, 83, 68, 3912, 15, 36, 165, 1539, 278, 36, 69, 2, 780, 8, 106, 14, 2, 1338, 18, 6, 22, 12, 215, 28, 610, 40, 6, 87, 326, 23, 2300, 21, 23, 22, 12, 272, 40, 57, 31, 11, 4, 22, 47, 6, 2307, 51, 9, 170, 23, 595, 116, 595, 1352, 13, 191, 79, 638, 89, 2, 14, 9, 8, 106, 607, 624, 35, 534, 6, 227, 7, 129, 113]
----label----
0


Note that the review is stored as a sequence of integers. These are word IDs that have been pre-assigned to individual words, and the label is an integer (0 for negative, 1 for positive).

# We can use the dictionary returned by imdb.get_word_index() to map the review back to the original words.

In [5]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

In [6]:
print('---review with words---')
print([id2word.get(i, ' ') for i in X_train[6]])
print('---label---')
print(y_train[1])

---review with words---
['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'overly', 'to', 'descent', 'people', 'time', 

# Maximum review length and minimum review length.

In [7]:
print('Minimum review length: {}'.format(
len(min((X_test + X_test), key=len))))

Minimum review length: 14


In [8]:
print('Maximum review length: {}'.format(
len(max((X_test + X_test), key=len))))

Maximum review length: 4630


# pad sequences

In [9]:
from keras.preprocessing import sequence

In [10]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [11]:
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]])

In [12]:
X_test

array([[   0,    0,    0, ...,   14,    6,  717],
       [   0,    0,    0, ...,  125,    4, 3077],
       [  33,    6,   58, ...,    9,   57,  975],
       ...,
       [   0,    0,    0, ...,   21,  846,    2],
       [   0,    0,    0, ..., 2302,    7,  470],
       [   0,    0,    0, ...,   34, 2005, 2643]])

# Design an RNN model for sentiment analysis

In [13]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [14]:
embedding_size = 24
model = Sequential()

In [15]:
model.add(Embedding(voc_size,embedding_size,input_length = max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 24)           120000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               50000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 170,101
Trainable params: 170,101
Non-trainable params: 0
_________________________________________________________________


# Train and evaluate our model

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
model.fit(X_train, y_train, validation_split=0.25, batch_size = 64, epochs=3)


Train on 18750 samples, validate on 6250 samples
Epoch 1/3
18750/18750 [==============================] - 219s 12ms/step - loss: 0.5257 - accuracy: 0.7179 - val_loss: 0.3863 - val_accuracy: 0.8318
Epoch 2/3
18750/18750 [==============================] - 252s 13ms/step - loss: 0.3300 - accuracy: 0.8633 - val_loss: 0.3691 - val_accuracy: 0.8386
Epoch 3/3
18750/18750 [==============================] - 296s 16ms/step - loss: 0.2730 - accuracy: 0.8905 - val_loss: 0.3341 - val_accuracy: 0.8629


In [19]:
scores = model.evaluate(X_test, y_test, verbose=0)

In [20]:
print("Test accuracy:", scores[2])

IndexError: list index out of range